# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score

In [4]:
# Display basic info
spaceship.head()

# Handle missing data
spaceship.fillna(method='ffill', inplace=True)

# Preprocessing
# Select numeric and categorical columns
numeric_features = spaceship.select_dtypes(include=['int64', 'float64']).columns
categorical_features = spaceship.select_dtypes(include=['object']).columns

# Define preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [6]:
# =========================================================
# 1. Importar librerías
# =========================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier
)
from sklearn.metrics import accuracy_score

# Quita estas dos líneas si tu scikit-learn es < 1.2
# from sklearn import set_config
# set_config(transform_output="pandas")

# =========================================================
# 2. Cargar el dataset
# =========================================================
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv"
spaceship = pd.read_csv(url)

# =========================================================
# 3. Manejo de datos faltantes
# =========================================================
# Rellenar valores faltantes con 'forward fill'
spaceship.fillna(method='ffill', inplace=True)

# =========================================================
# 4. Definir características numéricas y categóricas
# =========================================================
numeric_features = spaceship.select_dtypes(include=['int64', 'float64']).columns
categorical_features = spaceship.select_dtypes(include=['object']).columns

# =========================================================
# 5. ColumnTransformer para preprocesamiento
# =========================================================
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# =========================================================
# 6. Definir X e y
# =========================================================
X = spaceship.drop(columns=['Transported'])  
y = spaceship['Transported'].astype(int)

# =========================================================
# 7. Dividir en train y test
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# =========================================================
# 8. (Opcional) Probar con subset para debugging rápido
# =========================================================
# X_train = X_train.head(2000)
# y_train = y_train.head(2000)

# =========================================================
# 9. (Opcional) Revisar cuántas columnas produce la transformación
# =========================================================
X_preprocessed_sample = preprocessor.fit_transform(X_train)
print("Shape de X luego de la transformación:", X_preprocessed_sample.shape)

# =========================================================
# 10. Definir modelos con parámetros optimizados
# =========================================================

bagging_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', BaggingClassifier(
        base_estimator=RandomForestClassifier(
            n_estimators=10,
            random_state=42,
            n_jobs=-1
        ),
        n_estimators=10,
        random_state=42
    ))
])

pasting_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', BaggingClassifier(
        base_estimator=RandomForestClassifier(
            n_estimators=10,
            random_state=42,
            n_jobs=-1
        ),
        n_estimators=10,
        bootstrap=False,
        random_state=42
    ))
])

random_forest_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=50,
        random_state=42,
        n_jobs=-1
    ))
])

gradient_boosting_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(
        n_estimators=50,
        random_state=42
    ))
])

adaptive_boosting_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', AdaBoostClassifier(
        n_estimators=50,
        random_state=42
    ))
])

# =========================================================
# 11. Entrenar y evaluar los modelos
# =========================================================
models = {
    'Bagging': bagging_model,
    'Pasting': pasting_model,
    'Random Forest': random_forest_model,
    'Gradient Boosting': gradient_boosting_model,
    'Adaptive Boosting': adaptive_boosting_model
}

results = {}

for model_name, model in models.items():
    print(f"\n=== Entrenando {model_name}... ===")
    model.fit(X_train, y_train)
    print(f"{model_name} entrenado. Realizando predicciones...")
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy
    print(f"{model_name} Accuracy: {accuracy:.4f}")

# =========================================================
# 12. Seleccionar mejor modelo
# =========================================================
best_model = max(results, key=results.get)
print(f"\nEl mejor modelo es {best_model} con una exactitud de {results[best_model]:.4f}")


Shape de X luego de la transformación: (6954, 19238)

=== Entrenando Bagging... ===
Bagging entrenado. Realizando predicciones...
Bagging Accuracy: 0.7740

=== Entrenando Pasting... ===
Pasting entrenado. Realizando predicciones...
Pasting Accuracy: 0.7775

=== Entrenando Random Forest... ===
Random Forest entrenado. Realizando predicciones...
Random Forest Accuracy: 0.7746

=== Entrenando Gradient Boosting... ===
Gradient Boosting entrenado. Realizando predicciones...
Gradient Boosting Accuracy: 0.7775

=== Entrenando Adaptive Boosting... ===
Adaptive Boosting entrenado. Realizando predicciones...
Adaptive Boosting Accuracy: 0.7706

El mejor modelo es Pasting con una exactitud de 0.7775


- Evaluate your model

In [7]:
#your code here
# ============================================
# Supongamos que bagging_model es tu pipeline
# ============================================
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    # Hiperparámetros de BaggingClassifier
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_samples': [0.5, 0.8, 1.0],
    
    # Hiperparámetros del RandomForest interno
    'classifier__base_estimator__n_estimators': [10, 50],
    'classifier__base_estimator__max_depth': [None, 5, 10]
}

# Crear el objeto de búsqueda
grid_search = GridSearchCV(
    estimator=bagging_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Ejecutar la búsqueda en el conjunto de entrenamiento
grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("\nMejor score en validación cruzada (cv=5):")
print(grid_search.best_score_)

# Predecir con el mejor modelo encontrado
best_bagging = grid_search.best_estimator_
y_pred = best_bagging.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy en test con el modelo optimizado:", final_accuracy)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


In [1]:
#your code here

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [13]:
# =============================================
# 1. Importar librerías
# =============================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier
)
from sklearn.metrics import accuracy_score

# Quita o comenta las siguientes líneas si tu scikit-learn es < 1.2
# from sklearn import set_config
# set_config(transform_output="pandas")

# =============================================
# 2. Cargar y preparar el dataset
# =============================================
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv"
spaceship = pd.read_csv(url)

# Manejo de nulos
spaceship.fillna(method='ffill', inplace=True)

# Definir numéricas y categóricas
numeric_features = spaceship.select_dtypes(include=['int64', 'float64']).columns
categorical_features = spaceship.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

X = spaceship.drop(columns=['Transported'])  
y = spaceship['Transported'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# =============================================
# 3. Definir el pipeline del mejor modelo (Pasting)
# =============================================
pasting_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', BaggingClassifier(
        base_estimator=RandomForestClassifier(
            n_estimators=10,
            random_state=42,
            n_jobs=-1
        ),
        n_estimators=10,
        bootstrap=False,   # La diferencia de "pasting" vs "bagging"
        random_state=42
    ))
])

# =============================================
# 4. Definir rejilla de hiperparámetros
#    (Se ajustan tanto parámetros del Bagging
#     como del RandomForest base)
# =============================================
param_grid = {
    # Hiperparámetros del BaggingClassifier
    'classifier__n_estimators': [5, 10, 20],         # Cuántos "bags"
    'classifier__max_samples': [0.5, 1.0],          # Tamaño de muestreo
    'classifier__max_features': [0.5, 1.0],         # Proporción de features
    
    # Hiperparámetros del RandomForest base
    'classifier__base_estimator__n_estimators': [5, 10, 50],
    'classifier__base_estimator__max_depth': [None, 5, 10],
}

# =============================================
# 5. Hacer la búsqueda de hiperparámetros
# =============================================
grid_search = GridSearchCV(
    pasting_model, 
    param_grid=param_grid, 
    cv=5,             # 5-Fold Cross-validation
    scoring='accuracy',
    n_jobs=-1         # Paralelizar
)

print("\n=== Buscando mejores hiperparámetros para Pasting... ===")
grid_search.fit(X_train, y_train)

# =============================================
# 6. Imprimir mejores hiperparámetros
# =============================================
print("\nMejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# =============================================
# 7. Reentrenar y evaluar con esos parámetros
# =============================================
best_pasting = grid_search.best_estimator_
y_pred = best_pasting.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy con el mejor modelo de Pasting: {accuracy:.4f}")



=== Buscando mejores hiperparámetros para Pasting... ===

Mejores hiperparámetros encontrados:
{'classifier__base_estimator__max_depth': None, 'classifier__base_estimator__n_estimators': 50, 'classifier__max_features': 1.0, 'classifier__max_samples': 1.0, 'classifier__n_estimators': 20}

Accuracy con el mejor modelo de Pasting: 0.7803


- Run Grid Search

- Evaluate your model